In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Construct and print the path
functions_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'functions'))
print("Adding to sys.path:", functions_path)

# Add to sys.path if not already there
if functions_path not in sys.path:
    sys.path.append(functions_path)

# Print files to verify
print("Files in functions folder:", os.listdir(functions_path))

# Now try importing
import prospect_optimizer as po

Adding to sys.path: /Users/oliver/Desktop/GitHub/speciale_repo/functions
Files in functions folder: ['prospect_optimizer.py', 'GammaFunctions.py', 'expected_returns_puller.py', '__pycache__', 'conditionalAssetPricingLogMarginalLikelihoodTauClass.py', 'writeProfLatexTable.py', 'conditionalAssetPricingOOSPrediction.py', 'ConditionalAssetPricingCommonFunction.py', 'tictoc.py', 'CommonFunctions.py']


In [2]:
#Requirements
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize
import pandas as pd
import statsmodels.api as sm
from scipy.stats import mannwhitneyu, levene, f_oneway

import os
import sys
import time
import pickle as pkl


# Add the functions folder to the Python path
functions_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'functions'))
print("Files in functions folder:", os.listdir(functions_path))
print(functions_path)
import prospect_optimizer as po

Files in functions folder: ['prospect_optimizer.py', 'GammaFunctions.py', 'expected_returns_puller.py', '__pycache__', 'conditionalAssetPricingLogMarginalLikelihoodTauClass.py', 'writeProfLatexTable.py', 'conditionalAssetPricingOOSPrediction.py', 'ConditionalAssetPricingCommonFunction.py', 'tictoc.py', 'CommonFunctions.py']
/Users/oliver/Desktop/GitHub/speciale_repo/functions


In [3]:
homeDir = r'C:\Users\Tor Osted\OneDrive\Dokumenter\GitHub\speciale_repo\Complemetary Code Files for Submission\\'
dataDir = homeDir + r'Data\\'

n_predictors_to_use = 2 
assert 1 <= n_predictors_to_use <= 13, "You must choose between 1 and 13 predictors"

factors = pd.read_csv(dataDir + 'factors-20.csv')
factors = factors.drop(columns=['MKTRF','SMB*','MKT','CON','IA', 'ROE', 'ME'])  # remove redundant/duplicated
factorsNames = factors.columns.drop('Date')
for col in factors.columns:
   if col != 'Date':
        factors[col] = factors[col]


# === Step 5: Load and clean predictors ===
predictors = pd.read_csv(dataDir + 'Z - 197706.csv')
if 'Unnamed: 0' in predictors.columns:
    predictors = predictors.drop(columns='Unnamed: 0')

# Define full predictor index list
full_predictor_list = predictors.columns.drop('Date')
predictors[full_predictor_list] = predictors[full_predictor_list] / 100
significantPredictors = np.arange(n_predictors_to_use)
predictorsNames = full_predictor_list[significantPredictors]

# Subset actual predictor data
predictors = predictors[['Date'] + predictorsNames.tolist()]

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Tor Osted\\OneDrive\\Dokumenter\\GitHub\\speciale_repo\\Complemetary Code Files for Submission\\\\Data\\\\factors-20.csv'

In [ ]:

# === Step 6: Truncate all to shared sample size and index ===
start_date = '1997-01-01'
end_date = '2006-12-01'
date_index = pd.date_range(start=start_date, end=end_date, freq='MS')
n = len(date_index)

#returns = returns.iloc[:n].copy()

#returns_sim = returns_sim.iloc[:n].copy()

factors = factors.iloc[:n].copy()
predictors = predictors.iloc[:n].copy()
# === Convert decimal returns to percent returns ===

#returns.index = date_index
factors.index = date_index
predictors.index = date_index

# === Step 7: Reset index and rename for model compatibility ===
#returns_reset = returns.reset_index().rename(columns={'index': 'Date'})
#returns_reset.drop(columns='Row Labels', inplace=True) 

#returns_sim_reset = returns_sim.reset_index().rename(columns={'index': 'Date'})

factors_reset = factors.reset_index().rename(columns={'index': 'Date'})
predictors_reset = predictors.reset_index().rename(columns={'index': 'Date'})


#returns_reset   = returns.reset_index().rename(columns={'index': 'Date'})
factors_reset   = factors.reset_index().rename(columns={'index': 'Date'})
predictors_reset = predictors.reset_index().rename(columns={'index': 'Date'})

In [ ]:
bma_returns = po.calculate_bma_returns(0, factors_reset, predictors_reset, 1.1)

Tau= 1.100000 
Date    2006-03-01 00:00:00
Date                 198608
Name: 110, dtype: object
Date    2006-03-01 00:00:00
Date               8/1/1986
Name: 110, dtype: object
Market SR^2 in the estimation period= 0.021816
ZEstimation mean and std
[-0.0304918 -0.0304101]
[0.00136542 0.00133847]
ZTest mean and std
[-0.03450558 -0.0342835 ]
[0.00075889 0.00072605]
After demeaning
ZEstimation mean and std
[5.34925639e-15 1.79250554e-15]
[1. 1.]
ZTest mean and std
[-2.93959451 -2.89390993]
[0.55579342 0.54244947]
REstimation.shape= (0, 0)
RTest.shape= (0, 0)
FEstimation.shape= (110, 14)
FTest.shape= (9, 14)
ZEstimation.shape= (110, 2)
ZTest.shape= (9, 2)
KMax= 14, MMax= 2, T= 110, keyConditionalAPM=1
***** conditionalAssetPricingLogMarginalLikelihoodTauNumba **** 
Model  0 out of 65536
Model  6553 out of 65536
Model  13106 out of 65536
Model  19659 out of 65536
Model  26212 out of 65536
Model  32765 out of 65536
Model  39318 out of 65536
Model  45871 out of 65536
Model  52424 out of 65536

In [ ]:
len(bma_returns)

10000

In [ ]:

toc = time.time()

parent_dir = os.getcwd() # speciale_repo

### Input parameters
r_hat = 0.0  # Reference return
lambda_ = 1.5  # Base loss aversion coefficient
strategies = ["conservative","aggressive"]  # You can switch between "aggressive" or "conservative"
#lambda_values = [2.5, 1]
#gamma_values = [0.5, 2]

# strategies = ["conservative","aggressive"]
lambda_values = [0.5, 1, 1.5, 2, 2.5]

gamma_values = [0.12,0.2,0.25,0.35,0.5]

#Indlæser returnsdata
#datapath = os.path.join(parent_dir+'/data/returns_data.csv')

# Short data
datapath = os.path.join(parent_dir+'/data/returns.csv')


returns = pd.read_csv(datapath, index_col='Date')

# factors_result = po.optimize_portfolio(r_s, r_hat, lambda_, strategy)


returns_result = po.optimize_portfolio(bma_returns, r_hat, lambda_, gamma = 0.5)
print("Results are generating")
#results = po.resultgenerator(lambda_values, gamma_values, returns, strategies)
print("Done")



# Create returns_conservative
#returns_conservative = {
#    key: value for key, value in results.items() if "conservative" in key
#}

# Create returns_aggressive
#returns_aggressive = {
#    key: value for key, value in results.items() if "aggressive" in key
#}




tic = time.time()
print(f"Elapsed time: {(tic-toc):.2f}")

Results are generating
Done
Elapsed time: 14.13


In [ ]:
np.set_printoptions(suppress=True)
returns_result.x

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.06262763, 0.00020191, 0.16324801, 0.03811204, 0.42613962,
       0.05845232, 0.14415744, 0.0442445 , 0.06281652])